In [ ]:
import pydicom
import pymedphys
import numpy as np

reference_filepath = pymedphys.data_path("original_dose_beam_4.dcm") # reference (e.g., calculated) dose
evaluation_filepath = pymedphys.data_path("logfile_dose_beam_4.dcm") # evaluation (e.g., measured) dose

reference = pydicom.dcmread(reference_filepath)
evaluation = pydicom.dcmread(evaluation_filepath)

In [ ]:
axes_reference, dose_reference = pymedphys.dicom.zyx_and_dose_from_dataset(reference)
axes_evaluation, dose_evaluation = pymedphys.dicom.zyx_and_dose_from_dataset(evaluation)

dose_percent_threshold = 1
distance_mm_threshold = 1

In [ ]:
import matplotlib.pyplot as plt

z_ref, y_ref, x_ref = axes_reference

dose_difference = dose_evaluation - dose_reference

max_diff = np.max(np.abs(dose_difference))

# get the z-slice with the maximum dose difference
z_max_diff, _, _ = np.unravel_index(np.argmax(np.abs(dose_difference), axis=None), dose_difference.shape)

# we consider 10 z-slices above and below the maximum dose difference
z_start = z_max_diff - 10
z_end = z_max_diff + 10

fig, axes = plt.subplots(figsize=(15,10), nrows=4, ncols=5, sharex=True, sharey=True)
ax = axes.ravel().tolist()
ax[0].invert_yaxis() # invert just once as y axes are shared

for i, dose_diff_slice in enumerate(dose_difference[z_start:z_end]):

    im = ax[i].contourf(x_ref, y_ref, dose_diff_slice, 100, cmap=plt.get_cmap("seismic"), vmin=-max_diff, vmax=max_diff)
    ax[i].set_title(f"Slice Z_{z_start + i}")
    
    if i >= 15:
        ax[i].set_xlabel("x (mm)")
    
    if i % 5 == 0:
        ax[i].set_ylabel("y (mm)")

fig.tight_layout()
cbar = fig.colorbar(im, ax=ax, label="[Dose Eval] - [Dose Ref] (Gy)", aspect=40)
cbar.formatter.set_powerlimits((0, 0))

plt.show()

In [ ]:
%%time

gamma = pymedphys.gamma(
    axes_reference, dose_reference, 
    axes_evaluation, dose_evaluation, 
    dose_percent_threshold, distance_mm_threshold, interpolator="scipy")

In [ ]:
%%time

gamma = pymedphys.gamma(
    axes_reference, dose_reference, 
    axes_evaluation, dose_evaluation, 
    dose_percent_threshold, distance_mm_threshold, interpolator="econforge")